In [6]:
%pip install --upgrade datasets ragas langchain-openai unstructured "unstructured[md]" libmagic  python-magic python-magic-bin  

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: datasets in c:\sw\anaconda3\envs\torch\lib\site-packages (3.1.0)
  Using cached ragas-0.2.3-py3-none-any.whl.metadata (7.9 kB)
Using cached ragas-0.2.3-py3-none-any.whl (141 kB)
  Attempting uninstall: ragas
    Found existing installation: ragas 0.1.16
    Uninstalling ragas-0.1.16:
      Successfully uninstalled ragas-0.1.16



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ragas-haystack 1.0.1 requires ragas<=0.1.16,>=0.1.11, but you have ragas 0.2.3 which is incompatible.


In [2]:
import os
from getpass import getpass
os.environ["AZURE_OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")

In [3]:
OPENAI_DEPLOYMENT = 'gpt-35-turbo'
OPENAI_MODEL_NAME = 'gpt-35-turbo'

os.environ["OPENAI_API_VERSION"] = "2024-03-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://ciopenai.openai.azure.com/"


In [4]:
from datasets import load_dataset
from ragas import EvaluationDataset
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, SemanticSimilarity
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import AzureChatOpenAI, AzureOpenAI, AzureOpenAIEmbeddings


In [5]:
dataset = load_dataset("explodinggradients/amnesty_qa","english_v3", trust_remote_code=True)
eval_dataset = EvaluationDataset.from_hf_dataset(dataset["eval"])

Repo card metadata block was not found. Setting CardData to empty.


In [30]:
llm = AzureChatOpenAI(deployment_name="gpt-4o")
embeddings_llm = AzureOpenAIEmbeddings(deployment="text-embedding-ada-002")
evaluator_llm = LangchainLLMWrapper(llm)
evaluator_embeddings = LangchainEmbeddingsWrapper(embeddings_llm)

# Run the LLM
llm.invoke("Tell me a joke")

AIMessage(content="Sure, here's a joke for you:\n\nWhy don't skeletons fight each other?\n\nThey don't have the guts!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 11, 'total_tokens': 33, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_67802d9a6d', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'low'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'low'}}}, id='run-c20dee85-5eab-45e0-8932-a9b2f2541

In [31]:
metrics = [
    LLMContextRecall(llm=evaluator_llm), 
    FactualCorrectness(llm=evaluator_llm), 
    Faithfulness(llm=evaluator_llm),
    SemanticSimilarity(embeddings=evaluator_embeddings)
]
results = evaluate(dataset=eval_dataset, metrics=metrics, llm=llm, embeddings=embeddings_llm)

Evaluating:   0%|          | 0/80 [00:00<?, ?it/s]

In [32]:
df = results.to_pandas()
df

,user_input,retrieved_contexts,response,reference,context_recall,factual_correctness,faithfulness,semantic_similarity
0,What are the global implications of the USA Su...,"[- In 2022, the USA Supreme Court handed down ...",The global implications of the USA Supreme Cou...,The global implications of the USA Supreme Cou...,1.0,0.38,0.480000,0.959244
1,Which companies are the main contributors to G...,"[In recent years, there has been increasing pr...","According to the Carbon Majors database, the m...","According to the Carbon Majors database, the m...",1.0,0.11,0.120000,0.941763
2,Which private companies in the Americas are th...,[The issue of greenhouse gas emissions has bec...,"According to the Carbon Majors database, the l...",The largest private companies in the Americas ...,1.0,0.26,0.000000,0.959363
3,What action did Amnesty International urge its...,"[In the case of the Ogoni 9, Amnesty Internati...",Amnesty International urged its supporters to ...,Amnesty International urged its supporters to ...,1.0,0.25,0.600000,0.926831
4,What are the recommendations made by Amnesty I...,"[In recent years, Amnesty International has fo...",Amnesty International made several recommendat...,The recommendations made by Amnesty Internatio...,1.0,0.07,0.033333,0.919207
5,Who are the target audience of the two books c...,"[In addition to children, parents, teachers, a...",The target audience of the two books created b...,The target audience of the two books created b...,1.0,0.80,0.500000,0.987077
6,Which right guarantees access to comprehensive...,[The right to truth is a fundamental human rig...,The right that guarantees access to comprehens...,The right that guarantees access to comprehens...,1.0,0.91,1.000000,0.993913
7,Who has the right to be fully informed about h...,"[In many cases, the identities of perpetrators...",Everyone has the right to be fully informed ab...,The victims of gross human rights violations a...,1.0,0.45,0.340909,0.942685
8,When can individuals be found guilty under Art...,[Article 207.3 of the Russian Criminal Code pe...,Under Article 207.3 of the Russian Criminal Co...,Individuals can be found guilty under Article ...,1.0,0.00,0.000000,0.911983
9,When does the prosecution consider statements ...,[- As long as their statements are contrary to...,Under Article 207.3 of the Russian Criminal Co...,The prosecution considers statements contrary ...,1.0,0.00,0.000000,0.951724


Test set generator

In [52]:
!git clone https://huggingface.co/datasets/explodinggradients/Sample_Docs_Markdown

Cloning into 'Sample_Docs_Markdown'...


In [17]:
from langchain_community.document_loaders import DirectoryLoader

path = "Sample_Docs_Markdown/"
loader = DirectoryLoader(path, glob="**/*.md")
docs = loader.load()

In [23]:
docs

[Document(metadata={'source': 'Sample_Docs_Markdown\\1701.06538v1.md'}, page_content='Outrageously Large Neural Networks: The Sparsely-Gated Mixture-Of-Experts Layer\n\nNoam Shazeer1, Azalia Mirhoseini∗†1, Krzysztof Maziarz∗2, Andy Davis1, Quoc Le1, Geoffrey Hinton1and Jeff Dean1 1Google Brain, {noam,azalia,andydavis,qvl,geoffhinton,jeff}@google.com 2Jagiellonian University, Cracow, krzysztof.maziarz@student.uj.edu.pl\n\nAbstract\n\nThe capacity of a neural network to absorb information is limited by its number of parameters. Conditional computation, where parts of the network are active on a per-example basis, has been proposed in theory as a way of dramatically increasing model capacity without a proportional increase in computation. In practice, however, there are significant algorithmic and performance challenges. In this work, we address these challenges and finally realize the promise of conditional computation, achieving greater than 1000x improvements in model capacity with onl

In [26]:
from ragas.testset import TestsetGenerator

generator_llm = TestsetGenerator(llm=llm, embedding_model=embeddings_llm)

In [21]:
dataset = generator_llm.generate_with_langchain_docs(docs, testset_size=1)
dataset.to_pandas()


Applying [SummaryExtractor, HeadlinesExtractor]:   0%|          | 0/10 [00:00<?, ?it/s]

unable to apply transformation: object of type 'StringPromptValue' has no len()
unable to apply transformation: object of type 'StringPromptValue' has no len()
unable to apply transformation: object of type 'StringPromptValue' has no len()
unable to apply transformation: object of type 'StringPromptValue' has no len()
unable to apply transformation: object of type 'StringPromptValue' has no len()
unable to apply transformation: object of type 'StringPromptValue' has no len()
unable to apply transformation: object of type 'StringPromptValue' has no len()
unable to apply transformation: object of type 'StringPromptValue' has no len()
unable to apply transformation: object of type 'StringPromptValue' has no len()
unable to apply transformation: object of type 'StringPromptValue' has no len()


Applying EmbeddingExtractor:   0%|          | 0/5 [00:00<?, ?it/s]

unable to apply transformation: node.property('summary') must be a string, found '<class 'NoneType'>'
unable to apply transformation: node.property('summary') must be a string, found '<class 'NoneType'>'
unable to apply transformation: node.property('summary') must be a string, found '<class 'NoneType'>'
unable to apply transformation: node.property('summary') must be a string, found '<class 'NoneType'>'
unable to apply transformation: node.property('summary') must be a string, found '<class 'NoneType'>'


Applying HeadlineSplitter:   0%|          | 0/5 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying [EmbeddingExtractor, KeyphrasesExtractor, TitleExtractor]:   0%|          | 0/15 [00:00<?, ?it/s]

unable to apply transformation: object of type 'StringPromptValue' has no len()
unable to apply transformation: object of type 'StringPromptValue' has no len()
unable to apply transformation: object of type 'StringPromptValue' has no len()
unable to apply transformation: object of type 'StringPromptValue' has no len()
unable to apply transformation: object of type 'StringPromptValue' has no len()
unable to apply transformation: object of type 'StringPromptValue' has no len()
unable to apply transformation: object of type 'StringPromptValue' has no len()
unable to apply transformation: object of type 'StringPromptValue' has no len()
unable to apply transformation: object of type 'StringPromptValue' has no len()
unable to apply transformation: object of type 'StringPromptValue' has no len()


Applying CosineSimilarityBuilder:   0%|          | 0/1 [00:00<?, ?it/s]

ValueError: Node 4981bad8-ae7d-4f9b-8096-59ec931cf839 has no summary_embedding

Knowledge Graph


In [22]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()

In [24]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )

In [28]:
from ragas.testset.transforms import default_transforms, apply_transforms

# define your LLM and Embedding Model
# here we are using the same LLM and Embedding Model that we used to generate the testset
transformer_llm = generator_llm
embedding_model = embeddings_llm

trans = default_transforms(llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, trans)

Applying [SummaryExtractor, HeadlinesExtractor]:   0%|          | 0/10 [00:00<?, ?it/s]

unable to apply transformation: TestsetGenerator.generate() got an unexpected keyword argument 'n'
unable to apply transformation: TestsetGenerator.generate() got an unexpected keyword argument 'n'
unable to apply transformation: TestsetGenerator.generate() got an unexpected keyword argument 'n'
unable to apply transformation: TestsetGenerator.generate() got an unexpected keyword argument 'n'
unable to apply transformation: TestsetGenerator.generate() got an unexpected keyword argument 'n'
unable to apply transformation: TestsetGenerator.generate() got an unexpected keyword argument 'n'
unable to apply transformation: TestsetGenerator.generate() got an unexpected keyword argument 'n'
unable to apply transformation: TestsetGenerator.generate() got an unexpected keyword argument 'n'
unable to apply transformation: TestsetGenerator.generate() got an unexpected keyword argument 'n'
unable to apply transformation: TestsetGenerator.generate() got an unexpected keyword argument 'n'


Applying EmbeddingExtractor:   0%|          | 0/5 [00:00<?, ?it/s]

unable to apply transformation: node.property('summary') must be a string, found '<class 'NoneType'>'
unable to apply transformation: node.property('summary') must be a string, found '<class 'NoneType'>'
unable to apply transformation: node.property('summary') must be a string, found '<class 'NoneType'>'
unable to apply transformation: node.property('summary') must be a string, found '<class 'NoneType'>'
unable to apply transformation: node.property('summary') must be a string, found '<class 'NoneType'>'


Applying HeadlineSplitter:   0%|          | 0/5 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying [EmbeddingExtractor, KeyphrasesExtractor, TitleExtractor]:   0%|          | 0/15 [00:00<?, ?it/s]

unable to apply transformation: TestsetGenerator.generate() got an unexpected keyword argument 'n'
unable to apply transformation: TestsetGenerator.generate() got an unexpected keyword argument 'n'
unable to apply transformation: TestsetGenerator.generate() got an unexpected keyword argument 'n'
unable to apply transformation: TestsetGenerator.generate() got an unexpected keyword argument 'n'
unable to apply transformation: TestsetGenerator.generate() got an unexpected keyword argument 'n'
unable to apply transformation: TestsetGenerator.generate() got an unexpected keyword argument 'n'
unable to apply transformation: TestsetGenerator.generate() got an unexpected keyword argument 'n'
unable to apply transformation: TestsetGenerator.generate() got an unexpected keyword argument 'n'
unable to apply transformation: TestsetGenerator.generate() got an unexpected keyword argument 'n'
unable to apply transformation: TestsetGenerator.generate() got an unexpected keyword argument 'n'


Applying CosineSimilarityBuilder:   0%|          | 0/1 [00:00<?, ?it/s]

ValueError: Node a1d5895a-f4b3-4b94-bfa5-03a2a3600042 has no summary_embedding